# Import relevant libraries
- The convert_to_pm25 function was provided by Flowminder(Wilson, 2016b)

In [1]:
import os
import gdal
import glob
import datetime
from convert_to_pm25 import convert_to_pm25

## Specify variables

In [2]:
input_folder= r'D:\MAIAC_Data\Europe\h00v01'
output_folder = r'D:\MAIAC_Data\Europe\h00v01\Projected'
glob_string = '????\MAIAC?AOT*.hdf'
eta_Monthly = r'D:\Annies_Dissertation\MAIAC_Test\vanDs\eta_Monthly.tif'

# Translate Lat and Lon files
- Find out information about the lat lon file
- Using the gdal_translate tool convert the lat and lon hdr files into vrt format

In [ ]:
latlon_info = gdal.Info(r'D:\MAIAC_Data\Europe\MAIACLatlon.h00v01.hdf')
print(latlon_info)

gdal.Translate(r"D:\MAIAC_Data\Europe\h00v01\h00v01_lat.vrt", 
               r'HDF4_EOS:EOS_GRID:"D:\MAIAC_Data\Europe\MAIACLatlon.h00v01.hdf":latlon:lat', format = 'VRT' )

gdal.Translate(r"D:\MAIAC_Data\Europe\h00v01\h00v01_lon.vrt", 
               r'HDF4_EOS:EOS_GRID:"D:\MAIAC_Data\Europe\MAIACLatlon.h00v01.hdf":latlon:lon', format = 'VRT' )

# Warp the AOT images
- get a list of names of MAIAC AOT HDF files (the original MAIAC AOT files provided by Lyapustin et al 2011(a)
- Create the geolocation vrt file which contains all of the information needed to geolocate and warp the AOT images 
- Geolocate and warp function 
    - open and write the geolocation vrt file for each file name in list 
    - Perform the geolocate using the geolocation vrt file and warp to new specified coordinate system
- Create a month function which will enable the correct monthly van Donkelaar et al (2010) PM2.5 conversion factor to be applied to the AOT data
- Create a for statement which will loop over each file to 
    - Geolocate and warp AOT images
    - Convert AOT to PM2.5 using function provided by the Flowminder HOTBAR project using van Donkelaar et al (2010) PM2.5 conversion factors
    - Save warped AOT and PM2.5 images to a new folder which contains subfolders for each different year


In [5]:
# List of MAIAC AOT filenames
filenames = glob.glob(os.path.join(input_folder, glob_string))

In [3]:
# Geolocation.vrt file
geoloc_xml = """<VRTDataset rasterXSize="1200" rasterYSize="1200">
  <Metadata domain="GEOLOCATION">
     <mdi key="X_DATASET">h00v01_lon.vrt</mdi>
     <mdi key="X_BAND">1</mdi>
     <mdi key="Y_DATASET">h00v01_lat.vrt</mdi>
     <mdi key="Y_BAND">1</mdi>
     <mdi key="PIXEL_OFFSET">0</mdi>
     <mdi key="LINE_OFFSET">0</mdi>
     <mdi key="PIXEL_STEP">1</mdi>
     <mdi key="LINE_STEP">1</mdi>
  </Metadata>
  <VRTRasterBand dataType="Float32" band="1">
    <ColorInterp>Gray</ColorInterp>
    <SimpleSource>
      <SourceFilename relativeToVRT="0">HDF4_EOS:EOS_GRID:"{fname}":grid1km:Optical_Depth_Land</SourceFilename>
      <SourceBand>1</SourceBand>
      <SourceProperties RasterXSize="1200" RasterYSize="1200" DataType="Float32" BlockXSize="1200" BlockYSize="1200" />
      <SrcRect xOff="0" yOff="0" xSize="1200" ySize="1200" />
      <DstRect xOff="0" yOff="0" xSize="1200" ySize="1200" />
    </SimpleSource>
  </VRTRasterBand>
</VRTDataset>
"""

In [4]:
# Function to geolocate and warp the AOT images to British National Grid
def geolocate_and_warp(input_filename, output_filename, output_crs='EPSG:27700'):
    with open(r'geoloc_test.vrt', 'w') as myfile:
        myfile.write(geoloc_xml.format(fname=input_filename))
    gdal.Warp(output_filename,
             r'geoloc_test.vrt',
             geoloc = True, srcSRS = 'EPSG:4326', dstSRS = output_crs, 
              srcNodata = -28672, dstNodata = -28672)

In [6]:
# Function to specify month of AOT file to be used in conversion from AOT to PM2.5 
# using van Donkelaar et al (2010) monthly conversion factors
def month(AOT_filename):
       
    time_str = os.path.basename(AOT_filename)[17:28]
    time_obj = datetime.datetime.strptime(time_str, '%Y%j%H%M')
    mth = time_obj.month
    int_mth = int(mth)

    return int_mth

In [ ]:
# Loop over each MAIAC AOT file to produce geolocation.vrt with correct filename
# warp the image into new coordinate system 
# and convert AOT to PM2.5 
for filename in filenames:
    print ('Processing {fname}'.format(fname=filename))
    output_basename = os.path.join(output_folder,
                                  os.path.relpath(filename,
                                                 start = input_folder))
    output_dir = os.path.dirname(output_basename)
    AOT_filename = output_basename + "_proj.tif"
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    geolocate_and_warp(filename, AOT_filename)
    
    convert_to_pm25(AOT_filename, eta_Monthly, month(AOT_filename), 
                    output_basename + "_PM25.tif", 0.001)

Processing D:\MAIAC_Data\Europe\h00v01\2000\MAIACTAOT.h00v01.20000551145.hdf
Processing D:\MAIAC_Data\Europe\h00v01\2000\MAIACTAOT.h00v01.20000551325.hdf
Processing D:\MAIAC_Data\Europe\h00v01\2000\MAIACTAOT.h00v01.20000551500.hdf
Processing D:\MAIAC_Data\Europe\h00v01\2000\MAIACTAOT.h00v01.20000561050.hdf
Processing D:\MAIAC_Data\Europe\h00v01\2000\MAIACTAOT.h00v01.20000561405.hdf
Processing D:\MAIAC_Data\Europe\h00v01\2000\MAIACTAOT.h00v01.20000571135.hdf
Processing D:\MAIAC_Data\Europe\h00v01\2000\MAIACTAOT.h00v01.20000571315.hdf
Processing D:\MAIAC_Data\Europe\h00v01\2000\MAIACTAOT.h00v01.20000571450.hdf
Processing D:\MAIAC_Data\Europe\h00v01\2000\MAIACTAOT.h00v01.20000581040.hdf
Processing D:\MAIAC_Data\Europe\h00v01\2000\MAIACTAOT.h00v01.20000581355.hdf
Processing D:\MAIAC_Data\Europe\h00v01\2000\MAIACTAOT.h00v01.20000591125.hdf
Processing D:\MAIAC_Data\Europe\h00v01\2000\MAIACTAOT.h00v01.20000591300.hdf
Processing D:\MAIAC_Data\Europe\h00v01\2000\MAIACTAOT.h00v01.20000591435.hdf